In [9]:
# ✅ 1️⃣ Imports
from tensorflow.keras.applications import EfficientNetB1
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [10]:
# ✅ 2️⃣ Parameters
IMG_SIZE = (240, 240)
BATCH_SIZE = 32
DATA_DIR = '/home/deepak/Downloads/dataset/w/RealWaste/'

In [11]:
# ✅ 3️⃣ Data Preparation
datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

In [12]:
train_ds = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

Found 3804 images belonging to 9 classes.


In [13]:
val_ds = datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

Found 949 images belonging to 9 classes.


In [14]:
# ✅ 4️⃣ Build Model
base_model = EfficientNetB1(weights='imagenet', include_top=False, input_shape=(240, 240, 3))

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(train_ds.num_classes, activation='softmax')
])

2025-07-12 05:42:54.574462: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [15]:
# ✅ 5️⃣ Compile Model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [16]:
# ✅ 6️⃣ Callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ModelCheckpoint('best_waste_classifier.h5', monitor='val_loss', save_best_only=True)
]


In [ ]:
# ✅ 7️⃣ Train Model (Initial Training)
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=20,
    callbacks=callbacks
)

/home/deepak/anaconda3/lib/python3.11/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 0s 5s/step - accuracy: 0.5083 - loss: 1.6353

119/119 ━━━━━━━━━━━━━━━━━━━━ 740s 6s/step - accuracy: 0.5093 - loss: 1.6319 - val_accuracy: 0.1538 - val_loss: 2.4640
Epoch 2/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 633s 5s/step - accuracy: 0.7797 - loss: 0.6420 - val_accuracy: 0.1654 - val_loss: 2.9107
Epoch 3/20
119/119 ━━━━━━━━━━━━━━━━━━━━ 648s 5s/step - accuracy: 0.8303 - loss: 0.5306 - val_accuracy: 0.0917 - val_loss: 4.4094
Epoch 4/20
  4/119 ━━━━━━━━━━━━━━━━━━━━ 7:42 4s/step - accuracy: 0.8066 - loss: 0.7038

In [ ]:
# ✅ 8️⃣ OPTIONAL: Fine-tune (Unfreeze top layers)
base_model.trainable = True

In [ ]:
# ✅ Re-compile with lower learning rate
from tensorflow.keras.optimizers import Adam
model.compile(
    optimizer=Adam(1e-5),  # 👈 Smaller LR for fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# ✅ Fine-tune
history_finetune = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,  # Extra epochs for fine-tuning
    callbacks=callbacks
)


In [ ]:
print("✅ Training complete! Best model saved as `best_waste_classifier.h5`")